In [15]:
import pandas as pd
from datetime import datetime
import json

In [16]:
clients = {}
df = pd.read_csv(
    "metricscontribution_rate_initiative_cross_client_data_1613_for_students.csv")
clients_ids = df.client_id.unique()
for client_id in client_ids:
    client_id = str(client_id)
    clients.update(dict({client_id: {}}))

# print(clients)

{'1352108802': {}, '89281547': {}, '339825126': {}, '742770953': {}, '377235927': {}, '182523248': {}, '730912930': {}, '1890430896': {}, '146585525': {}, '1434827787': {}, '1588923963': {}, '1914596938': {}, '1991713172': {}, '930615366': {}, '962653': {}, '457842566': {}, '722889138': {}, '3045573': {}, '160484': {}, '141188634': {}, '1747449748': {}, '280972987': {}, '1343786340': {}, '1842186254': {}, '20028': {}, '1371350591': {}, '1094173963': {}, '2038648932': {}, '247049371': {}, '1222305620': {}, '44548903': {}, '1774320619': {}, '60253197': {}, '1039272198': {}, '232323': {}, '1999564604': {}, '1700828332': {}, '2024025171': {}, '1683841519': {}, '1444792769': {}, '883524406': {}, '360285558': {}, '470065716': {}, '1887255116': {}, '674982001': {}, '1348048562': {}, '1662358249': {}, '447268576': {}, '1167515548': {}, '204907870': {}, '601976963': {}, '957010757': {}, '2035193665': {}, '1288402200': {}, '32535': {}, '848061486': {}, '203634036': {}, '1816143665': {}, '5465034

In [39]:
for client_id in client_ids:
    client_specific_df = df[df["client_id"] == client_id][[
        "period_end", "source", "source_type", "attribution", "customers"]]

    periods = client_specific_df['period_end'].unique()

    # Update the dictionary for the client with periods
    period_dict = {}
    for period in periods:
        period_data = {}
        # Find the nr of attributed and unattributed customers
        period_df = client_specific_df[(
            client_specific_df['period_end'] == period)]
        attributed_customers = period_df[period_df['attribution']
                                         == 'ATTRIBUTED']['customers'].sum()
        unattributed_customers = period_df[period_df['attribution']
                                           != 'ATTRIBUTED']['customers'].sum()
        # Update the period data with the count of ATTRIBUTED and UNATTRIBUTED count
        # and the contribution rate for this period
        period_data["period"] = period
        period_data["ATTRIBUTED"] = int(attributed_customers)
        period_data["UNATTRIBUTED"] = int(unattributed_customers)
        period_data["CR"] = (attributed_customers / (attributed_customers + unattributed_customers) * 100)

        # Find and all the records for the period
        records = []
        sources = period_df['source'].unique()
        for source in period_df['source'].unique():
            source_df = period_df[period_df['source'] == source]
            source_data = {
                'source': str(source),  # Include source name
                'source_type': str(source_df.iloc[0]['source_type']),
                'ATTRIBUTED': source_df['attribution'].eq('ATTRIBUTED').any(),
                'UNATTRIBUTED': source_df['attribution'].ne('ATTRIBUTED').any(),
                'customers': int(source_df['customers'].sum())
            }
            records.append(source_data)

        period_data["records"] = records
        # Append the period data to the list for this client
        period_dict[period] = period_data

    # Add everything to the final dictionary for this client
    clients[str(client_id)] = period_dict


In [40]:
print(clients["2074305340"]["2023-02-12T00:00:00-08:00"])
file_path = "clients.json"
json_string = json.dumps(clients)
with open(file_path, 'w') as f:
    f.write(json_string)

{'period': '2023-02-12T00:00:00-08:00', 'ATTRIBUTED': 1, 'UNATTRIBUTED': 38, 'CR': 2.564102564102564, 'records': [{'source': 'direct', 'source_type': 'link', 'ATTRIBUTED': False, 'UNATTRIBUTED': True, 'customers': 38}, {'source': 'friend_landing_experience', 'source_type': 'link', 'ATTRIBUTED': True, 'UNATTRIBUTED': False, 'customers': 1}]}


TypeError: Object of type bool_ is not JSON serializable